In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
import time
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [ ]:
train_fname = '../data/train_cat_1.csv'
test_fname = '../data/test_cat_1.csv'
df = pd.read_csv(train_fname, sep=';')
df_test = pd.read_csv(test_fname, sep=';')
n_samples,n_variables = df.shape

In [ ]:
f_cat = ['VOIE_DEPOT','COUNTRY','SOURCE_BEGIN_MONTH','FISRT_APP_COUNTRY','FISRT_APP_TYPE','LANGUAGE_OF_FILLING',
        'FIRST_CLASSE','TECHNOLOGIE_SECTOR','TECHNOLOGIE_FIELD','MAIN_IPC','FISRT_INV_COUNTRY','FISRT_INV_TYPE','SOURCE_CITED_AGE',
        'SOURCE_IDX_ORI','SOURCE_IDX_RAD']

In [ ]:
y_fname = '../data/y_train_1.csv'
df_y = pd.read_csv(y_fname, sep=';')

In [ ]:
X_train = df[f_cat].values
X_test = df_test[f_cat].values
y_train = df_y['VARIABLE_CIBLE'].values == 'GRANTED'

In [ ]:
n_taken = n_samples
clf = RandomForestClassifier(max_depth=8)
start=time.time()
clf.fit(X_train[0:n_taken], y_train[0:n_taken])
print(time.time() - start)
y_pred_train = clf.predict_proba(X_train)[:, 1]
print('Score (optimiste) sur le train : %s' % roc_auc_score(y_train, y_pred_train))

In [ ]:
n_taken = n_samples
nb_thread = 4
clf = RandomForestClassifier()
parameters = {'n_estimators':[500],'criterion':['gini'],'max_depth':[170],'oob_score':[True],'min_samples_leaf':[5],'min_weight_fraction_leaf':[0],'min_samples_split':[2]}
def scorer(estimator, X, y):
    return roc_auc_score(y, estimator.predict_proba(X)[:,1])
gscv = GridSearchCV(clf, parameters, scorer, n_jobs = nb_thread)
start = time.time()
gscv.fit(X_train[0:n_taken],y_train[0:n_taken])
print(time.time() - start)
#y_pred_valid = gscv.predict_proba(X_train[n_taken:])[:, 1]
print('Score raisonnable sur le train : {}'.format(gscv.best_score_))
#print('Score sur la validation : {}'.format(roc_auc_score(y_train[n_taken:], y_pred_valid)))
print(gscv.best_params_)

In [ ]:
# 0.666286162905 {'oob_score': True, 'min_samples_leaf': 5, 'n_estimators': 30, 'min_weight_fraction_leaf': 0,
#                 'min_samples_split': 2, 'max_depth': 14}

# 0.667316737876 {'oob_score': True, 'min_samples_leaf': 5, 'n_estimators': 60, 'min_weight_fraction_leaf': 0,
#                 'min_samples_split': 2, 'max_depth': 14}

# 0.669209325129 {'oob_score': True, 'min_samples_leaf': 5, 'n_estimators': 200, 'min_weight_fraction_leaf': 0,
#                 'criterion': 'gini', 'min_samples_split': 2, 'max_depth': 50}

# 0.669389611075 {'oob_score': True, 'min_samples_leaf': 5, 'n_estimators': 500, 'min_weight_fraction_leaf': 0,
#                 'criterion': 'gini', 'min_samples_split': 2, 'max_depth': 50}

# 0.669437592563 {'oob_score': True, 'min_samples_leaf': 5, 'n_estimators': 500, 'min_weight_fraction_leaf': 0, 'criterion': 'gini',
#                 'min_samples_split': 2, 'max_depth': 90}

# 0.669496310911 {'oob_score': True, 'min_samples_leaf': 5, 'n_estimators': 500, 'min_weight_fraction_leaf': 0, 'criterion': 'gini',
#                 'min_samples_split': 2, 'max_depth': 170}

In [ ]:
n_taken = n_samples
nb_thread = 4
clf = RandomForestClassifier()
parameters = {'max_depth':[13,14,15],'oob_score':[False,True],'min_samples_leaf':[4,5,6],'min_weight_fraction_leaf':[0,0.1],'min_samples_split':[1,2,3]}
def scorer(estimator, X, y):
    return roc_auc_score(y, estimator.predict_proba(X)[:,1])
gscv = GridSearchCV(clf, parameters, scorer, n_jobs = nb_thread)
start = time.time()
gscv.fit(X_train[0:n_taken],y_train[0:n_taken])
print(time.time() - start)
#y_pred_valid = gscv.predict_proba(X_train[n_taken:])[:, 1]
print('Score raisonnable sur le train : {}'.format(gscv.best_score_))
#print('Score sur la validation : {}'.format(roc_auc_score(y_train[n_taken:], y_pred_valid)))
print(gscv.best_params_)

In [ ]:
# 0.662614486246 {'min_samples_split': 2, 'oob_score': False, 'min_weight_fraction_leaf': 0, 'max_depth': 14, 'min_samples_leaf': 5}
# On obtient un score équivalent avec une validation.
# Soumission à 0.665499020495 : tout va bien.
# 0.663469562149 {'min_samples_split': 2, 'oob_score': True, 'min_weight_fraction_leaf': 0, 'max_depth': 14, 'min_samples_leaf': 5}

In [ ]:
y_pred_train = gscv.predict_proba(X_train)[:, 1]
pd.DataFrame(data=y_pred_train, columns=['VARIABLE_CIBLE']).to_csv(path_or_buf='../y_trained/cat_RF_1_29.02.26.txt', sep=';')

In [ ]:
y_test = gscv.predict_proba(X_test)[:, 1]
np.savetxt('../subs/cate_RF_1.txt', y_test, fmt='%s')

#### Testons un truc stylé : On utilise les leaves des arbres créés par random forest comme features d'une régression logistique.

In [ ]:
X_train_rf, X_train_lr, y_train_rf, y_train_lr = train_test_split(X_train,
                                                            y_train,
                                                            test_size=0.5)

In [ ]:
#n_taken = n_samples
nb_thread = 4
clf = RandomForestClassifier()
parameters = {'n_estimators':[500],'criterion':['gini'],'max_depth':[170],'oob_score':[True],'min_samples_leaf':[5],'min_weight_fraction_leaf':[0],'min_samples_split':[2]}
def scorer(estimator, X, y):
    return roc_auc_score(y, estimator.predict_proba(X)[:,1])
gscv = GridSearchCV(clf, parameters, scorer, n_jobs = nb_thread)
start = time.time()
gscv.fit(X_train_rf,y_train_rf)
print(time.time() - start)
#y_pred_valid = gscv.predict_proba(X_train[n_taken:])[:, 1]
print('Score raisonnable de la CV : {}'.format(gscv.best_score_))
#print('Score sur la validation : {}'.format(roc_auc_score(y_train[n_taken:], y_pred_valid)))
print(gscv.best_params_)

In [ ]:
rf_enc = OneHotEncoder()
rf_enc.fit(gscv.best_estimator_.apply(X_train_rf))

nb_thread = 4
parameters = {'C':[1]}
clf = GridSearchCV(LogisticRegression(), parameters, scorer, n_jobs = nb_thread)
start = time.time()
clf.fit(rf_enc.transform(gscv.best_estimator_.apply(X_train_lr)), y_train_lr)
print(time.time() - start)
#y_pred_valid = gscv.predict_proba(X_train[n_taken:])[:, 1]
print('Score raisonnable de la CV : {}'.format(clf.best_score_))
#print('Score sur la validation : {}'.format(roc_auc_score(y_train[n_taken:], y_pred_valid)))
print(clf.best_params_)